<a href="https://colab.research.google.com/github/hunterhuang18/My-Dumb-Financial-Analysis/blob/main/Estimating_Total_Risk_and_Market_Risk_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

import warnings
warnings.filterwarnings('ignore')

In [5]:
appl = yf.download('AAPL', start='2010-01-01', end = '2013-12-31')
appl.head()

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2010-01-04 00:00:00+00:00,6.447412,7.643214,7.660714,7.585000,7.622500,493729600
2010-01-05 00:00:00+00:00,6.458558,7.656429,7.699643,7.616071,7.664286,601904800
2010-01-06 00:00:00+00:00,6.355826,7.534643,7.686786,7.526786,7.656429,552160000
2010-01-07 00:00:00+00:00,6.344076,7.520714,7.571429,7.466071,7.562500,477131200
2010-01-08 00:00:00+00:00,6.386255,7.570714,7.571429,7.466429,7.510714,447610800


##1. Estimating The Total Risk of a Stock

In [6]:
var_apple = np.var(appl['Adj Close'], ddof = 1)
std_apple = np.sqrt(var_apple)

##2. Estimating The Market Risk of a Stock

### 1. Estimating manually

In [7]:
# Download the SP500 index data
sp500 = yf.download('^GSPC', start = '2010-01-01', end = '2013-12-31')
sp500.head()

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,,
2010-01-04 00:00:00+00:00,1132.989990,1132.989990,1133.869995,1116.560059,1116.560059,3991400000
2010-01-05 00:00:00+00:00,1136.520020,1136.520020,1136.630005,1129.660034,1132.660034,2491020000
2010-01-06 00:00:00+00:00,1137.140015,1137.140015,1139.189941,1133.949951,1135.709961,4972660000
2010-01-07 00:00:00+00:00,1141.689941,1141.689941,1142.459961,1131.319946,1136.270020,5270680000
2010-01-08 00:00:00+00:00,1144.979980,1144.979980,1145.390015,1136.219971,1140.520020,4389590000


In [ ]:
# Combine the Adj Close of APPL and SP500
df = pd.concat([appl['Adj Close'], sp500['Adj Close']], axis = 1, keys = ['AAPL', 'SP500'])
df.head()

,AAPL,SP500
Date,,
2010-01-04,6.454506,1132.989990
2010-01-05,6.465664,1136.520020
2010-01-06,6.362820,1137.140015
2010-01-07,6.351057,1141.689941
2010-01-08,6.393281,1144.979980


In [ ]:
# Calculate the individual return for APPL and SP500
returns_df = df.pct_change(1)
returns_df.dropna(inplace = True)
new_col_names = ['AAPL_return', 'SP500_return']
returns_df.columns = new_col_names
returns_df.head()

,AAPL_return,SP500_return
Date,,
2010-01-05,0.001729,0.003116
2010-01-06,-0.015906,0.000546
2010-01-07,-0.001849,0.004001
2010-01-08,0.006648,0.002882
2010-01-11,-0.008821,0.001747


In [ ]:
# Calculate the deviation for APPL and SP500
dev_df = returns_df - returns_df.mean()
new_col_names = ['AAPL_dev', 'SP500_dev']
dev_df.columns = new_col_names
dev_df.head()

,AAPL_dev,SP500_dev
Date,,
2010-01-05,0.000593,0.002575
2010-01-06,-0.017042,0.000005
2010-01-07,-0.002984,0.003460
2010-01-08,0.005513,0.002341
2010-01-11,-0.009957,0.001206


In [ ]:
# Calculate the beta
cov = (dev_df['AAPL_dev'] * dev_df['SP500_dev']).sum()/(len(dev_df)-1)   #You can also use np.cov() to get the covariance between returns of AAPL and SP500
var_sp500 = np.var(returns_df['SP500_return'], ddof=1)
beta = cov/var_sp500
beta

0.8999738926901102

### 2. Estimating by Regression

In [ ]:
from scipy.stats import linregress

In [ ]:
linregress(y = returns_df['AAPL_return'], x = returns_df['SP500_return'])

LinregressResult(slope=0.8999738926901105, intercept=0.0006485939133897128, rvalue=0.5498703866844518, pvalue=2.0905517515722783e-80, stderr=0.04318691699225403, intercept_stderr=0.00046195053476439616)